In [1]:
import pandas as pd
from sqlalchemy import create_engine
from tqdm.auto import tqdm

In [2]:
db = dict(
    host='rostam.idav.ucdavis.edu',
    dbname='youtube',
    user='ytuser',
    passwd='GqBKuUigfQ4F0lyy'
)

def get_engine():
    return create_engine('mysql+pymysql://%s:%s@%s/%s' % (db['user'], db['passwd'], db['host'], db['dbname']))

In [3]:
puppets = pd.read_sql('sock-puppets', con=get_engine())

In [4]:
users = set()
items = set()

In [5]:
user_watch = []
user_recommend = []

In [6]:
for pup in puppets.itertuples():
    users.add(pup.puppet_id)
    for video in pup.viewed:
        items.add(video)
        user_watch.append((pup.puppet_id, video, 1))
    for trace in pup.recommendation_trail:
        for video in trace:
            items.add(video)
            user_watch.append((pup.puppet_id, video, 0))
    for video in pup.homepage:
        items.add(video)
        user_recommend.append((pup.puppet_id, video))

In [9]:
comments = pd.read_sql('SELECT author_id, video_id FROM comments', con=get_engine())
comments.to_csv('comments.csv', index=False)
comments = pd.read_csv('comments.csv')

In [12]:
comment_graph = []

In [13]:
for v1 in tqdm(items):
    for v2 in items:
        if v1 != v2:
            c1 = comments[comments['video_id'] == v1]
            c2 = comments[comments['video_id'] == v2]
            m = len(set(c1['author_id']) & set(c2['author_id']))
            comment_graph.append((v1, v2, m))

  0%|          | 0/127084 [00:00<?, ?it/s]

KeyboardInterrupt: 